In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key



In [2]:
city_data = pd.read_csv("output_data/cities.csv")
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Teano,41.2513,14.0665,62.92,90,7,2.82,IT,1666572003
1,Luanda,-8.8368,13.2343,75.79,85,98,7.38,AO,1666572003
2,Constantine,36.3650,6.6147,58.41,47,0,4.61,DZ,1666572004
3,Chui,-33.6971,-53.4616,56.61,95,77,6.26,UY,1666572004
4,Airai,-8.9266,125.4092,74.62,51,51,0.98,TL,1666572004


In [3]:
gmaps.configure(api_key = g_key)
locations = city_data[["Lat", "Lng"]].astype(float)
humidity = city_data["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
vacation_spots = city_data.loc[(city_data["Humidity"] < 65)
                                  & (city_data["Cloudiness"] < 15)
                                  & (city_data["Max Temp"] > 75) 
                                  & (city_data["Max Temp"] < 80)]

vacation_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,Muscat,23.6139,58.5922,77.02,45,0,2.37,OM,1666571772
150,Kirksville,40.1948,-92.5833,77.05,57,0,20.71,US,1666571865
189,Sur,22.5667,59.5289,78.33,38,0,7.27,OM,1666572128
202,Māngrol,21.1167,70.1167,78.48,57,0,4.23,IN,1666572134
400,Tete,-16.1564,33.5867,78.57,55,8,8.19,MZ,1666572222
439,Umm Kaddadah,13.6017,26.6876,79.07,20,11,20.58,SD,1666572240
492,Umluj,25.0213,37.2685,77.61,58,0,11.83,SA,1666572264
498,Jati,24.3539,68.2671,76.44,57,0,8.77,PK,1666572266
553,Nago,26.5881,127.9761,75.63,62,0,19.01,JP,1666572289


In [34]:
# lat = vacation_spots["Lat"]

# lng = vacation_spots["Lng"]

# lng

# vacation_spots.count(axis=1)

# coords = ["23.6139,58.5922","40.1948,-92.5833","22.5667,59.5289","21.1167,701167"]


In [5]:
# hotel_df = pd.DataFrame(vacation_spots)
hotel_df = vacation_spots.reset_index(drop = True)
hotel_df["Location"] = hotel_df["Lat"].astype(str) + "," + hotel_df["Lng"].astype(str)
hotel_df["Hotel Name"] = ""
hotel_df["Address"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Location,Hotel Name,Address
0,Muscat,23.6139,58.5922,77.02,45,0,2.37,OM,1666571772,"23.6139,58.5922",,
1,Kirksville,40.1948,-92.5833,77.05,57,0,20.71,US,1666571865,"40.1948,-92.5833",,
2,Sur,22.5667,59.5289,78.33,38,0,7.27,OM,1666572128,"22.5667,59.5289",,
3,Māngrol,21.1167,70.1167,78.48,57,0,4.23,IN,1666572134,"21.1167,70.1167",,
4,Tete,-16.1564,33.5867,78.57,55,8,8.19,MZ,1666572222,"-16.1564,33.5867",,
5,Umm Kaddadah,13.6017,26.6876,79.07,20,11,20.58,SD,1666572240,"13.6017,26.6876",,
6,Umluj,25.0213,37.2685,77.61,58,0,11.83,SA,1666572264,"25.0213,37.2685",,
7,Jati,24.3539,68.2671,76.44,57,0,8.77,PK,1666572266,"24.3539,68.2671",,
8,Nago,26.5881,127.9761,75.63,62,0,19.01,JP,1666572289,"26.5881,127.9761",,


In [6]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# hotel_df["Hotel"] = ""
# hotel_df["Address"] = ""

location = "23.6139,58.5922"

rankby = "distance"

type = "lodging"

radius = 5000

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
#     params["location"] = f"{lat},{lng}"

    
    params = {"radius": 5000,"type": "lodging", "key": g_key}
    
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params = params).json()
    
#     results = response["results"]
        
#         print(results)
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        hotel_df.loc[index, "Address"] = response["results"][0]["vicinity"]
    
    except: 
        print("Missing field... skipping")

# params = {"location": location, "rankby": rankby, "type": type,"radius": radius,"key": g_key}
# response = requests.get(base_url, params).json()
# # results = response["results"]
# response

# hotel_df

Missing field... skipping


In [7]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Location,Hotel Name,Address
0,Muscat,23.6139,58.5922,77.02,45,0,2.37,OM,1666571772,"23.6139,58.5922",Mutrah Hotel,"JH52+MVP, Muscat"
1,Kirksville,40.1948,-92.5833,77.05,57,0,20.71,US,1666571865,"40.1948,-92.5833",Budget Host-Village Inn,"1304 South Baltimore Street, Kirksville"
2,Sur,22.5667,59.5289,78.33,38,0,7.27,OM,1666572128,"22.5667,59.5289",Sur Plaza Hotel,Oman
3,Māngrol,21.1167,70.1167,78.48,57,0,4.23,IN,1666572134,"21.1167,70.1167",Hotel The Royal Honours,"43VQ+8FX, Mangrol - porbandar highway, vill. m..."
4,Tete,-16.1564,33.5867,78.57,55,8,8.19,MZ,1666572222,"-16.1564,33.5867",Zambeze Paraíso Misterioso,"Ponte Samora Machel, Tete"
5,Umm Kaddadah,13.6017,26.6876,79.07,20,11,20.58,SD,1666572240,"13.6017,26.6876",,
6,Umluj,25.0213,37.2685,77.61,58,0,11.83,SA,1666572264,"25.0213,37.2685",Moon light Furnished Units,"278C+XP4, Tabuk"
7,Jati,24.3539,68.2671,76.44,57,0,8.77,PK,1666572266,"24.3539,68.2671",Cafe Lala Rehan Restaurant,"9749+C55, Jati Road, Jati"
8,Nago,26.5881,127.9761,75.63,62,0,19.01,JP,1666572289,"26.5881,127.9761",Hotel Yugaf Inn Okinawa,"453-1 Miyazato, Nago"


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]



In [10]:
# coordinates = 

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(locations,info_box_content = hotel_info)

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))